<a href="https://colab.research.google.com/github/benschwarz3/Spring-2023/blob/main/Practice_AlexNet_Data_Train_Valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from google.colab import drive
from torchsummary import summary

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. Load labeled images from folders
# data_dir = '/content/drive/MyDrive/MAP Data 2023/Flowers'
data_dir = '/content/drive/MyDrive/MAP Data 2023/Animals'

In [ ]:
cd /content/drive/MyDrive/MAP Data 2023/Animals

/content/drive/MyDrive/MAP Data 2023/Animals


In [ ]:
ls

Train/  Valid/


In [ ]:
# 3. Pre-process the data and create data loaders
data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) for x in ['Train', 'Valid']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4) for x in ['Train', 'Valid']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['Train', 'Valid']}
class_names = image_datasets['Train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 4. Set up the AlexNet architecture
alexnet = models.alexnet(pretrained=True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:05<00:00, 48.1MB/s]


In [ ]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:

num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, len(class_names)) # Change last layer
alexnet = alexnet.to(device) # Put on GPU

In [ ]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
summary(alexnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
class_names

['Dogs', 'Wolves']

In [ ]:
dataset_sizes

{'Train': 42, 'Valid': 12}

In [ ]:
for inputs, labels in dataloaders["Train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

In [ ]:
inputs.shape

torch.Size([10, 3, 224, 224])

In [ ]:
labels.shape

torch.Size([10])

In [ ]:
labels

tensor([1, 0, 0, 0, 1, 1, 0, 1, 0, 1])

In [ ]:
outputs = alexnet(inputs)

In [ ]:
outputs.shape

torch.Size([10, 2])

In [ ]:
outputs

tensor([[-0.4258,  0.0312],
        [-0.1528, -0.2891],
        [-0.8919,  0.5216],
        [-1.3731,  0.0490],
        [-1.4323,  0.0624],
        [-0.8420, -0.1723],
        [-0.1410, -0.0114],
        [-0.2154,  0.2644],
        [ 0.1437,  0.1772],
        [ 0.0468, -0.0910]], grad_fn=<AddmmBackward0>)

In [ ]:
torch.max(outputs,1)[0]

tensor([ 0.0312, -0.1528,  0.5216,  0.0490,  0.0624, -0.1723, -0.0114,  0.2644,
         0.1772,  0.0468], grad_fn=<MaxBackward0>)

In [ ]:
torch.max(outputs,1)[1]

tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [ ]:
preds = torch.max(outputs, 1)[1]

In [ ]:
preds

tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [ ]:
labels

tensor([1, 0, 0, 0, 1, 1, 0, 1, 0, 1])

In [ ]:
preds == labels

tensor([ True,  True, False, False,  True,  True, False,  True, False, False])

In [ ]:
labels.shape[0]

10

In [ ]:
torch.sum(preds == labels)/labels.shape[0]

tensor(0.5000)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

# Train the AlexNet model
alexnet.train()

num_epochs = 20

for epoch in range(num_epochs):
    print(epoch, " of ", num_epochs - 1)
    print('-' * 10)

    running_corrects = 0

    for inputs, labels in dataloaders["Train"]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        

        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]



        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_corrects += torch.sum(preds == labels.data)

    print('Train Acc: {:.4f}'.format(running_corrects / dataset_sizes["Train"]))




    # Evaluate the AlexNet model on Validation Data
    alexnet.eval()

    running_corrects = 0

    for inputs, labels in dataloaders["Valid"]:
        inputs = inputs.to(device)
        labels = labels.to(device)


        outputs = alexnet(inputs)
        preds = torch.max(outputs, 1)[1]


        running_corrects += torch.sum(preds == labels.data)

    acc_valid = running_corrects / dataset_sizes["Valid"]
    print('Valid Acc: {:.4f}'.format(acc_valid))
    if acc_valid > 0.99:
        print("Done!")
        break


print('Training complete')

0  of  19
----------
Train Acc: 0.6429
Valid Acc: 0.7500
1  of  19
----------
Train Acc: 0.8571
Valid Acc: 1.0000
Done!
Training complete


In [ ]:
pwd

'/content/drive/MyDrive/MAP Data 2023/Animals'

In [ ]:
model_path = '/content/drive/MyDrive/MAP Data 2023/Animals/alexnet_classification_animals.pth'

In [ ]:
# Save the trained model
torch.save(alexnet.state_dict(), model_path)

In [ ]:
ls

alexnet_classification_animals.pth  Train/  Valid/


In [ ]:
import requests
from PIL import Image
from io import BytesIO

In [ ]:
# 1. Load the trained AlexNet model
def load_model(model_path):
    model = models.alexnet()
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model.to(device)

In [ ]:
# 2. Define a function to load an image from a URL and preprocess it
def preprocess_image(url, transform):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img_tensor = transform(img)
    return img_tensor.unsqueeze(0).to(device)

In [ ]:
# 3. Perform inference using the loaded model
def predict_image_url(url, model):
    img_tensor = preprocess_image(url, data_transforms['Valid'])
    output = model(img_tensor)
    pred = torch.max(output, 1)[1]
    return class_names[pred]

In [ ]:
trained_model = load_model(model_path)

In [ ]:
image_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcThC-k81_wHLClQS3P04BWiF_3jiIbV64SRkQ&usqp=CAU'

In [ ]:
prediction = predict_image_url(image_url, trained_model)

In [ ]:
print("The predicted class for the input image is:", prediction)

The predicted class for the input image is: Wolves
